In [1]:
%load_ext autoreload
%autoreload 2

This is a notebook that illustrates how to run the end-to-end (e2e) workflow. Determining suitable parameters for the workflow are for operational considerations are explained in the ops library [`dist-s1-enumerator`](https://github.com/opera-adt/dist-s1-enumerator) - specifically, see this [notebook](https://github.com/opera-adt/dist-s1-enumerator/blob/dev/notebooks/A__Staging_Inputs_for_One_MGRS_Tile.ipynb).

In [2]:
from pathlib import Path

from dist_s1.workflows import (
    run_burst_disturbance_workflow,
    run_dist_s1_localization_workflow,
    run_dist_s1_workflow,
    run_disturbance_merge_workflow,
    run_dist_s1_packaging_workflow,
    run_despeckle_workflow,
    run_dist_s1_sas_prep_workflow
)

/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
## Example 0

# mgrs_tile_id = '10SGD'
# post_date = '2025-01-02'
# track_number = 137
# dst_dir = Path('out')
# memory_strategy = 'high'


## Example 1 - Los Angeles Wildfire
mgrs_tile_id = '11SLT'
post_date = '2025-01-21'
track_number = 71
dst_dir = Path('los-angeles')
memory_strategy = 'high'
interpolation_method='bilinear'
n_workers_for_norm_param_estimation=1
stride_for_norm_param_estimation=16
model_compilation=True
device='cpu'
model_dtype = 'float32'

This takes some time as it not only generates the runconfig but also localizes the necessary data. It will not overwrite files that have been previously downloaded.

Few additional notes:

1. The runconfig can be serialized to yml and then read from that file.
2. The runconfig manages all the paths for the workflows from the initial set of inputs. It's data model is messy and can be confusing. But that's the workhorse of this library.

In [17]:
run_config = run_dist_s1_sas_prep_workflow(
    mgrs_tile_id,
    post_date,
    track_number,
    post_date_buffer_days=1,
    dst_dir=dst_dir,
    input_data_dir=dst_dir,
    interpolation_method=interpolation_method,
    n_workers_for_norm_param_estimation=n_workers_for_norm_param_estimation,
    device=device,
    stride_for_norm_param_estimation=stride_for_norm_param_estimation,
    model_dtype=model_dtype,
    model_compilation=False
)


Using multi-window lookback strategy


In [19]:
if run_config.algo_config.apply_despeckling:
    run_despeckle_workflow(run_config)

In [ ]:
# Compute disturbance per burst and all possible lookbacks
run_burst_disturbance_workflow(run_config)

Chips Traversed: 100%|██████████████████████████████████| 656/656 [00:23<00:00, 28.24it/s]
<string>:34: RuntimeWarning: All-NaN slice encountered
Chips Traversed:  83%|████████████████████████████▎     | 489/586 [00:16<00:03, 30.60it/s]

In [ ]:
# Merge the burst-wise products
run_disturbance_merge_workflow(run_config)

In [ ]:
run_dist_s1_packaging_workflow(run_config)